In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os,glob
import re

In [2]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/McLeod/McLeod complete data/Product master files")

In [ ]:
#Import file which has mfg name and part num for products
data_file=pd.read_excel("Access GUDID.xlsx")

In [ ]:
data_file=data_file[data_file.MNF_PART_NUM_PM.notnull()]

In [ ]:
#output data frame to collect scrapped results
final_output=pd.DataFrame()

In [ ]:
for idx,row in data_file.iterrows():
    mfg_part_num=str(row['MNF_PART_NUM_PM']) #extract mpn for the product
    mfg_name=str(row['MANUFACTURER_NAME_PM']) #extract mfg name for the product
    mfg_name=mfg_name.replace(" ",'+')
    #create a URL as shown below using mfg name and mpn
    URL="https://accessgudid.nlm.nih.gov/devices/search?query=versionModelNumber%3A%28%22"+mfg_part_num+"%22%29+AND+companyName%3A%28"+mfg_name+"%29"
    
    try:
        data=requests.get(URL)
    except:
        continue
    
    try:
        data=BeautifulSoup(data.text, 'html.parser')
    except:
        continue
    
    try:
        result_row=data.find_all('div',{'class':'resultRow'})
    except:
        continue
    
    try:
        #extract device id of the products which were returned as result by the server and store them
        devi_id_links=result_row[0].find_all('div',{'class':'xsmall-11'})[0].find_all('a',href=True)
    except:
        continue
    
    if(devi_id_links):
        for i in range(len(devi_id_links)):
            try:
                device_id=devi_id_links[i]['href']
            except:
                continue
            
            #Collect device id for each product along with SK. This device ID will be used to directly go to product page
            final_output=final_output.append({"Secondary_Key":row['Secondary_Key'],
                                             "DeviceID":device_id},ignore_index=True)
            print("Length of output: {}".format(len(final_output)))

In [ ]:
final_output.to_excel("Device_Id_file.xlsx")

In [3]:
#read the file with devie ID of the products
data=pd.read_excel("Device_Id_file.xlsx")

In [4]:
data.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [5]:
data.head()

,DeviceID,Secondary_Key
0,/devices/10885425323579,SK-7417
1,/devices/10885425323579,SK-7418
2,/devices/10885425323579,SK-7419
3,/devices/10884521013704,SK-7475
4,/devices/10884521013704,SK-7476


In [6]:
#output dataframe to store the scrapped values
sourced_data=pd.DataFrame()

In [ ]:
for idx,row in data.iterrows():
    URL="https://accessgudid.nlm.nih.gov"+str(row['DeviceID']) # create a url of product with device id
    label_dict={} #dict to store the device attribute label
    pkg_info_tbl={} #dict to store packaging info of product
    try:
        r=requests.get(URL)
        r=BeautifulSoup(r.text, 'html.parser')
    except:
        continue
    
    try:
        sections=r.find_all('div',{'class':'deviceInformation'})
        labels=sections[0].find_all('label',{'class':'device-attribute-label'})
        for label in labels:
            key=label.get_text()
            value=label.nextSibling.strip()
            label_dict.setdefault(key,[]).append(value)
    except:
        print("Issue 1")
        pass
    
    try:
        packaging=sections[0].find_all('table',{'class':'packageDItable'})
        pkg_tbl_headers=packaging[0].find_all('thead')[0].find_all('th')
        pkg_tbl_value_tr=packaging[0].find_all('tbody')[0].find_all('tr')
        for tr in pkg_tbl_value_tr:
            td_value_list=tr.find_all('td')
            for i in range(len(td_value_list)):
                value=td_value_list[i].get_text()
                key=pkg_tbl_headers[i].get_text()
                pkg_info_tbl.setdefault(key,[]).append(value)
    except:
        print("Issue 2")
        pass
    
    # label dict is appended with pkg info dict
    label_dict.update(pkg_info_tbl)
    label_dict['Secondary_key']=row['Secondary_Key']
    sourced_data=sourced_data.append(label_dict,ignore_index=True)
    print("Length of output:  {}".format(len(sourced_data)))

In [9]:
sourced_data.to_excel("sourced_data.xlsx")